In [ ]:
! cp "/content/drive/My Drive/negative.csv" "/content"
! cp "/content/drive/My Drive/positive.csv" "/content"

In [ ]:
!tf_upgrade_v2 — infile foo.py — outfile foo-upgraded.py

/bin/bash: tf_upgrade_v2 — infile: command not found


#1. Загрузка данных из текстовых файлов

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os, re, csv, math, codecs

n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['text'].values[:sample_size],
                           data_negative['text'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

In [ ]:
import re

def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()


data = [preprocess_text(t) for t in raw_data]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=2)

#2. Метрики качества классификации

In [ ]:
from keras import backend as K


def precision(y_true, y_pred):

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
       
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

Using TensorFlow backend.


#3 Векторизация слов

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

SENTENCE_LENGTH = 26
NUM = 100000

def get_sequences(tokenizer, x):
    sequences = tokenizer.texts_to_sequences(x)
    return pad_sequences(sequences, maxlen=SENTENCE_LENGTH)

tokenizer = Tokenizer(num_words=NUM)
tokenizer.fit_on_texts(x_train)

x_train_seq = get_sequences(tokenizer, x_train)
x_test_seq = get_sequences(tokenizer, x_test)

In [ ]:
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open('/content/fasttextr.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

2314it [00:00, 23137.10it/s]

loading word embeddings...


1386609it [01:03, 21923.89it/s]

found 1386609 word vectors


In [ ]:
DIM=100

#embedding matrix
print('preparing embedding matrix...')
words_not_found = []
word_index = tokenizer.word_index
NUM = min(NUM, len(word_index))
embedding_matrix = np.zeros((NUM, DIM))
for word, i in word_index.items():
    if i >= NUM: continue
    embedding_vector = embeddings_index.get(word)
    
# words not found in embedding index will be all-zeros.
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

preparing embedding matrix...
number of null word embeddings: 21079


# 4. Построение сверточной нейросети

In [ ]:
from keras.layers import Input
from keras.layers.embeddings import Embedding

tweet_input = Input(shape=(SENTENCE_LENGTH,), dtype='int32')
tweet_encoder = Embedding(NUM, DIM, input_length=SENTENCE_LENGTH,
                          weights=[embedding_matrix], trainable=False)(tweet_input)

In [ ]:
from keras import optimizers
from keras.layers import Dense, concatenate, Activation, Dropout
from keras.models import Model
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import GlobalMaxPooling1D

branches = []
x = Dropout(0.1)(tweet_encoder)

for size, filters_count in [(2,6), (3, 6)]:
    for i in range(filters_count):
        branch = Conv1D(filters=1, kernel_size=size, padding='valid', activation='sigmoid')(x)
        branch = GlobalMaxPooling1D()(branch)
        branches.append(branch)

x = concatenate(branches, axis=1)
x = Dropout(0.1)(x)
x = Dense(30, activation='relu')(x)
x = Dense(1)(x)
output = Activation('sigmoid')(x)

model = Model(inputs=[tweet_input], outputs=[output])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[precision, recall, f1])
model.summary()

W0613 18:37:58.964887 140247981520768 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0613 18:37:59.229192 140247981520768 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0613 18:37:59.253093 140247981520768 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 26)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 26, 100)      10000000    input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 26, 100)      0           embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 25, 1)        201         dropout_1[0][0]                  
__________________________________________________________________________________________________
conv1d_2 (

# 5 Обучение и оценка качества

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("cnn-frozen-embeddings-{epoch:02d}-{val_f1:.2f}.hdf5", 
                             monitor='val_f1', save_best_only=True, mode='max', period=1)
history = model.fit(x_train_seq, y_train, batch_size=32, epochs=10, validation_split=0.25, callbacks = [checkpoint])

Train on 134307 samples, validate on 44769 samples
Epoch 1/10
134307/134307 [==============================] - 47s 354us/step - loss: 0.6093 - precision: 0.6665 - recall: 0.6556 - f1: 0.6511 - val_loss: 0.5563 - val_precision: 0.7483 - val_recall: 0.6554 - val_f1: 0.6917
Epoch 2/10
134307/134307 [==============================] - 43s 323us/step - loss: 0.5652 - precision: 0.7094 - recall: 0.6950 - f1: 0.6939 - val_loss: 0.5361 - val_precision: 0.7617 - val_recall: 0.6637 - val_f1: 0.7020
Epoch 3/10
134307/134307 [==============================] - 43s 321us/step - loss: 0.5491 - precision: 0.7183 - recall: 0.7022 - f1: 0.7023 - val_loss: 0.5356 - val_precision: 0.8068 - val_recall: 0.5902 - val_f1: 0.6737
Epoch 4/10
134307/134307 [==============================] - 43s 320us/step - loss: 0.5396 - precision: 0.7267 - recall: 0.7063 - f1: 0.7089 - val_loss: 0.5293 - val_precision: 0.8007 - val_recall: 0.6104 - val_f1: 0.6849
Epoch 5/10
134307/134307 [==============================] - 43s 3

In [ ]:
model.load_weights('cnn-frozen-embeddings-08-0.73.hdf5')

In [ ]:
from sklearn.metrics import classification_report

predicted = np.round(model.predict(x_test_seq))
print(classification_report(y_test, predicted, digits=5))

              precision    recall  f1-score   support

           0    0.72373   0.78412   0.75271     22457
           1    0.76281   0.69874   0.72937     22313

    accuracy                        0.74157     44770
   macro avg    0.74327   0.74143   0.74104     44770
weighted avg    0.74320   0.74157   0.74108     44770



In [ ]:
from keras import optimizers

model.layers[1].trainable = True
adam = optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=[precision, recall, f1])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 26)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 26, 100)      10000000    input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 26, 100)      0           embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 25, 1)        201         dropout_1[0][0]                  
__________________________________________________________________________________________________
conv1d_2 (

In [ ]:
checkpoint = ModelCheckpoint("cnn-trainable-{epoch:02d}-{val_f1:.2f}.hdf5", 
                             monitor='val_f1', save_best_only=True, mode='max', period=1)

history_trainable = model.fit(x_train_seq, y_train, batch_size=32, epochs=10, validation_split=0.25, callbacks = [checkpoint])

Train on 134307 samples, validate on 44769 samples
Epoch 1/10
134307/134307 [==============================] - 75s 562us/step - loss: 0.4589 - precision: 0.7827 - recall: 0.7693 - f1: 0.7703 - val_loss: 0.4771 - val_precision: 0.8006 - val_recall: 0.7056 - val_f1: 0.7436
Epoch 2/10
134307/134307 [==============================] - 75s 560us/step - loss: 0.4566 - precision: 0.7867 - recall: 0.7708 - f1: 0.7730 - val_loss: 0.4743 - val_precision: 0.7953 - val_recall: 0.7166 - val_f1: 0.7474
Epoch 3/10
134307/134307 [==============================] - 75s 558us/step - loss: 0.4526 - precision: 0.7869 - recall: 0.7748 - f1: 0.7750 - val_loss: 0.4759 - val_precision: 0.8027 - val_recall: 0.7054 - val_f1: 0.7441
Epoch 4/10
134307/134307 [==============================] - 75s 558us/step - loss: 0.4483 - precision: 0.7923 - recall: 0.7778 - f1: 0.7795 - val_loss: 0.4726 - val_precision: 0.7976 - val_recall: 0.7182 - val_f1: 0.7493
Epoch 5/10
134307/134307 [==============================] - 75s 5

In [ ]:
model.load_weights('cnn-trainable-10-0.75.hdf5')

In [ ]:
predicted = np.round(model.predict(x_test_seq))
print(classification_report(y_test, predicted, digits=5))

              precision    recall  f1-score   support

           0    0.74672   0.80750   0.77592     22457
           1    0.78897   0.72433   0.75527     22313

    accuracy                        0.76605     44770
   macro avg    0.76784   0.76591   0.76559     44770
weighted avg    0.76777   0.76605   0.76563     44770

